In [1]:
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1714708387125_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, expr, sum, min, max, concat, to_date, length, datediff, lit, current_date
from pyspark.sql.window import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
redshift_url = "jdbc:redshift://{host}:{port}/{database}".format(
    host="pidiliteredshiftfractal.ckxxid8ylf5i.ap-south-1.redshift.amazonaws.com",
    port="5439",
    database="pidilite_db"
)

redshift_properties = {
    "user": "awsuser",
    "password": "Awstestuser123",
    "driver": "com.amazon.redshift.jdbc42.Driver"
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def date_filter_select_features(df, date_column, start_day, end_day, feature_columns):
    ## Change to date time
    df = df.withColumn(
    date_column, 
    to_date(col(date_column).cast("string"), "yyyyMMdd")
    )
    # Filter the DataFrame based on start day and end day
    filtered_df = df.filter((col(date_column) >= start_day) & (col(date_column) <= end_day))

    # Select the desired feature columns
    selected_df = filtered_df.select(*feature_columns)

    return selected_df
# def apply_aggregations(df, group_by_columns, aggregation_dict):
#     # Apply the specified aggregations to the DataFrame
#     aggregated_df = df.groupBy(*group_by_columns).agg(*aggregation_dict)

#     return aggregated_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
current_yearmomnth= datetime.now()
current_yearmomnth= datetime(current_yearmomnth.year, current_yearmomnth.month, current_yearmomnth.day)
current_yearmomnth

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datetime.datetime(2024, 5, 3, 0, 0)

In [7]:
current_yearmomnth.year, current_yearmomnth.month

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(2024, 5)

In [8]:
#dealer_master= spark.read.parquet("s3://pidilite-raw-zone/silver/Dimdealer_MS/")
# sales_group_to_filter= ['201', '301', '105', '401', '901', '902']
#dealer_master= dealer_master.filter(col('salesgroupcode').isin(sales_group_to_filter))
#dealer_master.count(), dealer_master.select('dealersgkey').distinct().count(), dealer_master.select('salesgroupcode').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h3> Input params

In [9]:
target_cap_n_times= 3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# invoice table
scheme_dryrun = spark.read.format("jdbc")\
                .option("url", redshift_url) \
                .option("query", """
                select *
                from pidilite_test.scheme_factdryrundetails
                """)\
                .option("user", redshift_properties['user']) \
                .option("password", redshift_properties['password']) \
                .load()

scheme_master = spark.read.format("jdbc")\
                .option("url", redshift_url) \
                .option("query", """
                select *
                from pidilite_test.scheme_FactSecondary
                """)\
                .option("user", redshift_properties['user']) \
                .option("password", redshift_properties['password']) \
                .load()

modality = spark.read.format("jdbc")\
                .option("url", redshift_url) \
                .option("query", """
                select *
                from pidilite_test.Scheme_FactSecondaryModality
                """)\
                .option("user", redshift_properties['user']) \
                .option("password", redshift_properties['password']) \
                .load()

# dealer master
dealer_master = spark.read.format("jdbc")\
                .option("url", redshift_url) \
                .option("query", """
                select dealersgkey, deriveddealergroupcode
                from pidilite_test.Dimdealer_MS
                """)\
                .option("user", redshift_properties['user']) \
                .option("password", redshift_properties['password']) \
                .load()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
scheme_dryrun.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- schemecode: long (nullable = true)
 |-- schemename: string (nullable = true)
 |-- startdate: timestamp (nullable = true)
 |-- enddate: timestamp (nullable = true)
 |-- customercode: string (nullable = true)
 |-- dealercode: string (nullable = true)
 |-- dealername: string (nullable = true)
 |-- cluster: string (nullable = true)
 |-- salesvolumely: decimal(18,2) (nullable = true)
 |-- salesvaluely: decimal(18,2) (nullable = true)
 |-- salesvolumecy: decimal(18,2) (nullable = true)
 |-- salesvaluecy: decimal(18,2) (nullable = true)
 |-- slab: integer (nullable = true)
 |-- growth: decimal(18,2) (nullable = true)
 |-- basepoints: decimal(18,2) (nullable = true)
 |-- bonuspoints: decimal(18,2) (nullable = true)
 |-- eligiblepoints: decimal(18,2) (nullable = true)
 |-- discounts: decimal(18,2) (nullable = true)
 |-- target: decimal(18,2) (nullable = true)
 |-- invoiceno: string (nullable = true)
 |-- giftitem: string (nullable = true)
 |-- giftquantity: decimal(18,2) (nullable = t

In [13]:
dealer_master.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- dealersgkey: string (nullable = true)
 |-- deriveddealergroupcode: string (nullable = true)

In [14]:
current_yearmomnth

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datetime.datetime(2024, 5, 3, 0, 0)

In [16]:
# filter active schemes and salesgroupcode
print(scheme_dryrun.count())
scheme_dryrun = scheme_dryrun.filter((col("enddate") >= current_yearmomnth))
#scheme_dryrun = scheme_dryrun.filter((col("salesgroupcode").isin(sales_group_to_filter)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50345

In [17]:
# drop duplicates on dealer and scheme level in dry run table
print(scheme_dryrun.count())
scheme_dryrun= scheme_dryrun.dropDuplicates(['dealercode', 'schemecode'])
print(scheme_dryrun.count())

# select all required features in scheme dry run table
scheme_dryrun= scheme_dryrun.select(['salesgroupcode', 'schemecode', 'customercode', 'dealercode', 'salesvolumely', 'salesvaluely', 'salesvolumecy', 'salesvaluecy', 'slab', 'growth', 'basepoints', 'bonuspoints', 'eligiblepoints', 'discounts'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50345
50255

In [18]:
# drop duplicates on other tables to merge on dry run table
scheme_master= scheme_master.dropDuplicates()
modality= modality.dropDuplicates(['id', 'slabno', 'modalityto'])

# merge scheme master for operated ON info, Modality target from Modality table
scheme_dryrun= scheme_dryrun.join(scheme_master.select(['id', 'fromdate', 'todate', 'operatedon', 'schemetitle']), scheme_master.id== scheme_dryrun.schemecode , 'left')
scheme_dryrun= scheme_dryrun.join(modality.select(['id', 'slabno', 'modalityto']), (scheme_dryrun.schemecode== modality.id) & (scheme_dryrun.slab== modality.slabno), how= 'left')

print(scheme_dryrun.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50255

In [21]:
# create derived columns
scheme_dryrun= scheme_dryrun.withColumn("UpperThresholdOfSlabVal", when((col("operatedon")== "Value"), col('modalityto'))) \
                            .withColumn("UpperThresholdOfSlabVol", when((col("operatedon")!= "Value"), col('modalityto')))\
                            .withColumn("Target", when((col("operatedon")== "Value"), col('UpperThresholdOfSlabVal')- col('salesvaluecy')).otherwise(col('UpperThresholdOfSlabVol')- col('salesvolumecy')))\
                            .withColumn("duration", datediff(col('todate'), col('fromdate')))\
                            .withColumn("ActiveRecommendation", when(col('Target')>0 , 1).otherwise(0))\
                            .withColumn('Activity', lit('Scheme'))\
                            .withColumn('dealersgkey', concat(col('dealercode'), col('salesgroupcode')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
'''
Update target- where the dealer is in last slab and upper threshold is not defined- placeholder values present in modality table
Condition checks:
1. check if upper threshold >80000    OR
2. Slab is undefined                 OR
1. If all 9's before decimal or otherwise + no of 9's before decimal >5 + achievement <50% of upper threshold

Modify target in those data points- 20% of current achivement

'''

scheme_dryrun = scheme_dryrun.withColumn("Target",
    when(((col("operatedon") == "Value") & (col('UpperThresholdOfSlabVal')>= 80000)) |
         ((col("operatedon") == "Value") &  (col('Slab').isNull())) |
         ((col("operatedon") == "Value") & 
          (length(regexp_replace(split(col('UpperThresholdOfSlabVal').cast('string'), '\.').getItem(0), "[^9]", "")) == length(split(col('UpperThresholdOfSlabVal').cast('string'), '\.').getItem(0))) & 
          (length(split(col('UpperThresholdOfSlabVal').cast('string'), '\.').getItem(0))> 5) &
          (col('salesvaluecy') <= 0.5 * col('UpperThresholdOfSlabVal'))),
         0.2 * col('salesvaluecy')
    )
    .when(((col("operatedon") != "Value") & (col('UpperThresholdOfSlabVol')>= 80000)) |
          ((col("operatedon") != "Value") & (col('Slab').isNull())) |
          ((col("operatedon") != "Value") & 
           (length(regexp_replace(split(col('UpperThresholdOfSlabVol').cast('string'), '\.').getItem(0), "[^9]", "")) == length(split(col('UpperThresholdOfSlabVol').cast('string'), '\.').getItem(0))) & 
           (length(split(col('UpperThresholdOfSlabVol').cast('string'), '\.').getItem(0))> 5) &
           (col('salesvolumecy') <= 0.5 * col('UpperThresholdOfSlabVol'))), 
        0.2 * col('salesvolumecy')
     )
    .otherwise(col('Target'))
)
print(scheme_dryrun.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50255

In [23]:
# even if now some high target, then cap target at 300% of current achievement
scheme_dryrun= scheme_dryrun.withColumn('Target', when((col("operatedon") == "Value") &
                                                       (col('Target')> target_cap_n_times*col('salesvaluecy')),
                                                       target_cap_n_times*col('salesvaluecy')
                                                      )
                                                .when((col("operatedon") != "Value") &
                                                       (col('Target')> target_cap_n_times*col('salesvolumecy')),
                                                       target_cap_n_times*col('salesvolumecy')
                                                      )
                                                .otherwise(col('Target'))
                                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
scheme_dryrun= scheme_dryrun.drop(*['modalityto', 'id', 'slabno'])
print(scheme_dryrun.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50255

In [25]:
scheme_dryrun.select('salesgroupcode').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|salesgroupcode|
+--------------+
|           401|
|           301|
|           302|
|           111|
|           105|
|           201|
|           406|
|           107|
+--------------+

In [26]:
scheme_dryrun= scheme_dryrun.join(dealer_master, on=['dealersgkey'], how= 'left')
scheme_dryrun.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50255

In [27]:
scheme_dryrun.select('salesgroupcode').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|salesgroupcode|
+--------------+
|           401|
|           301|
|           302|
|           111|
|           105|
|           201|
|           406|
|           107|
+--------------+

In [28]:
# Scheme Priority
scheme_dryrun= scheme_dryrun.withColumn("percent", when((col("operatedon")== "Value"), round(col('Target')/col('salesvaluecy'), 4)).otherwise(round(col('Target')/col('salesvolumecy'), 4)))

window_spec = Window.partitionBy("dealersgkey").orderBy(col("percent").desc())

scheme_dryrun = scheme_dryrun.withColumn("SchemePriority", row_number().over(window_spec))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Rename columns
scheme_rename_dict= {
'schemecode': 'SchemeCode',
'schemetitle': 'SchemeTitle',
'fromdate': 'StartDate',
'todate': 'EndDate',
'customercode': 'CustomerCode',
'deriveddealergroupcode': 'DealerGroupCode',
'dealersgkey': 'DealerSGCode',
'salesgroupcode': 'SalesGroupCode',
'dealercode': 'DealerCode',
'salesvolumely': 'SalesVolumeLY',
'salesvaluely': 'SalesValueLY',
'salesvolumecy': 'SalesVolumeCY',
'salesvaluecy': 'SalesValueCY',
'slab': 'Slab',
'growth': 'Growth',
'basepoints': 'BasePoints',
'bonuspoints': 'BonusPoints',
'eligiblepoints': 'EligiblePoints',
'discounts': 'Discounts',
'duration': 'Duration',
'operatedon': 'OperatedON'
}

for old_col, new_col in scheme_rename_dict.items():
    scheme_dryrun= scheme_dryrun.withColumnRenamed(old_col, new_col)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# Approximate decimal columns to nearest place 2
scheme_dryrun = scheme_dryrun.withColumn("SalesVolumeLY", round(col("SalesVolumeLY"), 2))\
                            .withColumn("SalesValueLY", round(col("SalesValueLY"), 2))\
                            .withColumn("SalesVolumeCY", round(col("SalesVolumeCY"), 2))\
                            .withColumn("SalesValueCY", round(col("SalesValueCY"), 2))\
                            .withColumn("Growth", round(col("Growth"), 2))\
                            .withColumn("BasePoints", round(col("BasePoints"), 2))\
                            .withColumn("BonusPoints", round(col("BonusPoints"), 2))\
                            .withColumn("EligiblePoints", round(col("EligiblePoints"), 2))\
                            .withColumn("Discounts", round(col("Discounts"), 2))\
                            .withColumn("UpperThresholdOfSlabVal", round(col("UpperThresholdOfSlabVal"), 2))\
                            .withColumn("UpperThresholdOfSlabVol", round(col("UpperThresholdOfSlabVol"), 2))\
                            .withColumn("Target", round(col("Target"), 2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
scheme_dryrun.select('salesgroupcode').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|salesgroupcode|
+--------------+
|           401|
|           301|
|           302|
|           111|
|           105|
|           201|
|           406|
|           107|
+--------------+

In [32]:
# Update load columns

load_quarter = (current_yearmomnth.month - 1) // 3 + 4 if current_yearmomnth.month in [1, 2, 3] else (current_yearmomnth.month - 1) // 3
load_year= (current_yearmomnth.year) if current_yearmomnth.month in [1, 2, 3] else current_yearmomnth.year + 1

scheme_dryrun= scheme_dryrun.withColumn('LoadQtr', concat(lit(load_year), lit('Q'), lit(load_quarter)))\
                            .withColumn('LoadMnth', lit(current_yearmomnth.month))\
                            .withColumn('LoadWeekNum', lit(current_yearmomnth.isocalendar()[1]))\
                            .withColumn('LoadFreq', lit('Weekly'))\
                            .withColumn('LoadDate', lit(current_yearmomnth))\
                            .withColumn('LoadDate', date_format(col('LoadDate'), 'yyyy-MM-dd 12:00:00.000000'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# Final order of columns
scheme_dryrun= scheme_dryrun.select('SchemeCode', 'SchemeTitle', 'StartDate', 'EndDate', 'CustomerCode', 'DealerGroupCode', 'DealerSGCode', 'SalesGroupCode', 'DealerCode', 'SalesVolumeLY', 'SalesValueLY', 'SalesVolumeCY', 'SalesValueCY', 'Slab', 'UpperThresholdOfSlabVol', 'UpperThresholdOfSlabVal', 'Target', 'Growth', 'BasePoints', 'BonusPoints', 'EligiblePoints', 'Discounts', 'Duration', 'OperatedON', 'SchemePriority', 'ActiveRecommendation', 'Activity', 'LoadQtr', 'LoadMnth', 'LoadWeekNum', 'LoadFreq', 'LoadDate')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
scheme_dryrun.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+-------------------+-------------------+------------+---------------+------------+--------------+----------+-------------+------------+-------------+------------+----+-----------------------+-----------------------+------+------+----------+-----------+--------------+---------+--------+----------+--------------+--------------------+--------+-------+--------+-----------+--------+--------------------+
|SchemeCode|         SchemeTitle|          StartDate|            EndDate|CustomerCode|DealerGroupCode|DealerSGCode|SalesGroupCode|DealerCode|SalesVolumeLY|SalesValueLY|SalesVolumeCY|SalesValueCY|Slab|UpperThresholdOfSlabVol|UpperThresholdOfSlabVal|Target|Growth|BasePoints|BonusPoints|EligiblePoints|Discounts|Duration|OperatedON|SchemePriority|ActiveRecommendation|Activity|LoadQtr|LoadMnth|LoadWeekNum|LoadFreq|            LoadDate|
+----------+--------------------+-------------------+-------------------+------------+---------------+------------+--------------+

In [35]:
scheme_dryrun.select('salesgroupcode').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|salesgroupcode|
+--------------+
|           401|
|           301|
|           302|
|           111|
|           105|
|           201|
|           406|
|           107|
+--------------+

<h3> Dtype conversion

In [36]:
from pyspark.sql import types as T
scheme_schema = T.StructType([
    T.StructField("SchemeCode", T.IntegerType(), False),
    T.StructField("SchemeTitle", T.StringType(), False),
    T.StructField("StartDate", T.StringType(), False),
    T.StructField("EndDate", T.StringType(), False),
    T.StructField("CustomerCode", T.IntegerType(), False),
    T.StructField("DealerGroupCode", T.StringType(), False),
    T.StructField("DealerSGCode", T.StringType(), False),
    T.StructField("SalesGroupCode", T.StringType(), False),
    T.StructField("DealerCode", T.StringType(), False),
    T.StructField("SalesVolumeLY", T.DecimalType(18,2), False),
    T.StructField("SalesValueLY", T.DecimalType(18,2), False),
    T.StructField("SalesVolumeCY", T.DecimalType(18,2), False),
    T.StructField("SalesValueCY", T.DecimalType(18,2), False),
    T.StructField("Slab", T.IntegerType(), False),
    T.StructField("UpperThresholdOfSlabVol", T.DecimalType(18,2), False),
     T.StructField("UpperThresholdOfSlabVal", T.DecimalType(18,2), False),
     T.StructField("Target", T.DecimalType(18,2), False),
     T.StructField("Growth", T.DecimalType(18,2), False),
     T.StructField("BasePoints", T.DecimalType(18,2), False),
     T.StructField("BonusPoints", T.DecimalType(18,2), False),
     T.StructField("EligiblePoints", T.DecimalType(18,2), False),
     T.StructField("Discounts", T.DecimalType(18,2), False),
    T.StructField("Duration", T.IntegerType(), False),
    T.StructField("OperatedON", T.StringType(), False),
    T.StructField("SchemePriority", T.IntegerType(), False),
    T.StructField("ActiveRecommendation", T.IntegerType(), False),
    T.StructField("Activity", T.StringType(), False),
    T.StructField("LoadQtr", T.StringType(), False),
    T.StructField("LoadMnth", T.IntegerType(), False),
    T.StructField("LoadWeekNum", T.IntegerType(), False),
    T.StructField("LoadFreq", T.StringType(), False),
    T.StructField("LoadDate", T.StringType(), False)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
for field in scheme_schema.fields:
    scheme_dryrun = scheme_dryrun.withColumn(field.name, scheme_dryrun[field.name].cast(field.dataType))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
scheme_dryrun.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- SchemeCode: integer (nullable = true)
 |-- SchemeTitle: string (nullable = true)
 |-- StartDate: string (nullable = true)
 |-- EndDate: string (nullable = true)
 |-- CustomerCode: integer (nullable = true)
 |-- DealerGroupCode: string (nullable = true)
 |-- DealerSGCode: string (nullable = true)
 |-- SalesGroupCode: string (nullable = true)
 |-- DealerCode: string (nullable = true)
 |-- SalesVolumeLY: decimal(18,2) (nullable = true)
 |-- SalesValueLY: decimal(18,2) (nullable = true)
 |-- SalesVolumeCY: decimal(18,2) (nullable = true)
 |-- SalesValueCY: decimal(18,2) (nullable = true)
 |-- Slab: integer (nullable = true)
 |-- UpperThresholdOfSlabVol: decimal(18,2) (nullable = true)
 |-- UpperThresholdOfSlabVal: decimal(18,2) (nullable = true)
 |-- Target: decimal(18,2) (nullable = true)
 |-- Growth: decimal(18,2) (nullable = true)
 |-- BasePoints: decimal(18,2) (nullable = true)
 |-- BonusPoints: decimal(18,2) (nullable = true)
 |-- EligiblePoints: decimal(18,2) (nullable = tr

<h3> Check nulls

In [39]:
scheme_dryrun.select([F.count(F.when(F.col(c).isNull() | F.isnan(c), c)).alias(c) for c in scheme_dryrun.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+---------+-------+------------+---------------+------------+--------------+----------+-------------+------------+-------------+------------+----+-----------------------+-----------------------+------+------+----------+-----------+--------------+---------+--------+----------+--------------+--------------------+--------+-------+--------+-----------+--------+--------+
|SchemeCode|SchemeTitle|StartDate|EndDate|CustomerCode|DealerGroupCode|DealerSGCode|SalesGroupCode|DealerCode|SalesVolumeLY|SalesValueLY|SalesVolumeCY|SalesValueCY|Slab|UpperThresholdOfSlabVol|UpperThresholdOfSlabVal|Target|Growth|BasePoints|BonusPoints|EligiblePoints|Discounts|Duration|OperatedON|SchemePriority|ActiveRecommendation|Activity|LoadQtr|LoadMnth|LoadWeekNum|LoadFreq|LoadDate|
+----------+-----------+---------+-------+------------+---------------+------------+--------------+----------+-------------+------------+-------------+------------+----+-----------------------+-----------------------

In [40]:
scheme_dryrun= scheme_dryrun.fillna(0, subset= ['Slab', 'UpperThresholdOfSlabVol', 'UpperThresholdOfSlabVal'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
scheme_dryrun.select([F.count(F.when(F.col(c).isNull() | F.isnan(c), c)).alias(c) for c in scheme_dryrun.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+---------+-------+------------+---------------+------------+--------------+----------+-------------+------------+-------------+------------+----+-----------------------+-----------------------+------+------+----------+-----------+--------------+---------+--------+----------+--------------+--------------------+--------+-------+--------+-----------+--------+--------+
|SchemeCode|SchemeTitle|StartDate|EndDate|CustomerCode|DealerGroupCode|DealerSGCode|SalesGroupCode|DealerCode|SalesVolumeLY|SalesValueLY|SalesVolumeCY|SalesValueCY|Slab|UpperThresholdOfSlabVol|UpperThresholdOfSlabVal|Target|Growth|BasePoints|BonusPoints|EligiblePoints|Discounts|Duration|OperatedON|SchemePriority|ActiveRecommendation|Activity|LoadQtr|LoadMnth|LoadWeekNum|LoadFreq|LoadDate|
+----------+-----------+---------+-------+------------+---------------+------------+--------------+----------+-------------+------------+-------------+------------+----+-----------------------+-----------------------

<h3> Save output

In [42]:
scheme_dryrun.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+-------------------+-------------------+------------+---------------+------------+--------------+----------+-------------+------------+-------------+------------+----+-----------------------+-----------------------+------+------+----------+-----------+--------------+---------+--------+----------+--------------+--------------------+--------+-------+--------+-----------+--------+--------------------+
|SchemeCode|         SchemeTitle|          StartDate|            EndDate|CustomerCode|DealerGroupCode|DealerSGCode|SalesGroupCode|DealerCode|SalesVolumeLY|SalesValueLY|SalesVolumeCY|SalesValueCY|Slab|UpperThresholdOfSlabVol|UpperThresholdOfSlabVal|Target|Growth|BasePoints|BonusPoints|EligiblePoints|Discounts|Duration|OperatedON|SchemePriority|ActiveRecommendation|Activity|LoadQtr|LoadMnth|LoadWeekNum|LoadFreq|            LoadDate|
+----------+--------------------+-------------------+-------------------+------------+---------------+------------+--------------+

In [43]:
scheme_dryrun = scheme_dryrun.orderBy(col('SalesGroupCode').asc(),col('DealerSGCode').asc())
scheme_dryrun.write.mode("overwrite").parquet("s3://pidilite-raw-zone/gold/scheme_achievement/schemes_updated/")
#scheme_dryrun.write.mode("overwrite").csv("s3://pidilite-raw-zone/gold/scheme_achievement/schemes_updated/", header= True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…